# CPTS 499

## Adding on past data to the dataframe (NO2, Ozone, Lead, CO, SO2, and some others)

## To use this script:

You need to first signup with AirData. They provide you with a key and email. It is a unique way to sign up. [Here it is](https://aqs.epa.gov/aqsweb/documents/data_api.html#signup).

With that done, you need to configure a `config.json` file, and set the `CONFIG_TO_JSON` path in the constants tab. The json file should look like:

```json
{
    ...
    "aqs_data" : {
        "email": "<your email here>",
        "key": "<your key here>"
    }
    ...
}
```

The next thing to consider is the `PATH_TO_CSV` in the constants tab. Make that the path to your CSV folder.

## Tips:

In the constants tab, you will see three variable: `PARTICULATE_MATTER`, `LONGITUDE_RANGE`, `LATITTUDE_RANGE`. These three are configurable value also. If you want more parameters for your data, visit [this link](https://aqs.epa.gov/aqsweb/documents/codetables/parameter_classes.html) to find more available data.

The latitude and longitude range is wide because some of these measurements are harder to find. Therefore, we make a larger box to ensure we find something for it.

The endpoint we call is set by box. You can either give the `GetAirData` just the latitude and longitude you want, or you can give it a min and max latitude and longitude. To use the latter option, the code would have to be a changed a little, as this reads the location from the CSV file. 

In [1]:
# install new dependcies
%pip install haversine


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# imports
import pandas as pd
import datetime as dt
import requests as req
import json
import math
from collections import Counter as ct
import haversine as hs

In [17]:
# constants
DATE_FORMAT = "%Y-%m-%d"
PATH_TO_CSV = "../gsur23013.csv"
PATH_TO_CONFIG = "../config.json"
AQS_DATA = {}
with open(PATH_TO_CONFIG, "r") as f:
        AQS_DATA = json.load(f)["aqs_data"]

# this url retrieves the daily data from the AirData services
# example: https://aqs.epa.gov/data/api/dailyData/byBox?email=your_email@email.com&key=you_key&param=44201,42101&bdate=20150501&edate=20150502&minlat=33.3&maxlat=33.6&minlon=-87.0&maxlon=-86.7
# for the query parameter "parameter", you can pass a comma-seperated list it to retrieve multiple different particulate matters. See the dictionary defined below
REQUEST_URL = "https://aqs.epa.gov/data/api/dailyData/byBox?email={}&key={}&param={}&bdate={}&edate={}&minlat={}&maxlat={}&minlon={}&maxlon={}"

PARTICULATE_MATTERS = {
        "Ozone" : 44201, 
        "Lead (TSP) LC" : 14129,
        "CO" : 42101,
        "SO2" : 42401,
        "NO2" : 42602,
        "PM10 Total 0-10um STP" : 81102,
        "Lead PM10 LC FRM/FEM" : 85129,
        "PM2.5 - Local Conditions" : 88101,
        }

LATITUDE_RANGE = 7.0
LONGITUDE_RANGE = 7.0

In [4]:
# load the csv and anything else
csv_data = pd.read_csv(PATH_TO_CSV, nrows=400000, skiprows=[1])

# change stamp column to datetime object

# Strip the date from a string
# params
#   date: the str representation of the date
# returns
#   a datetime object
def GetDate(date: str) -> dt.datetime:
    day = str.split(date, " ")[0]
        
    # strip out the datetime object .
    date = dt.datetime.strptime(day, DATE_FORMAT)
    return date

# Applies the GetDate function to the entire given series
# params
#   date: the series to apply the date to
# returns
#   the changed date column
def GetDateSeries(date: pd.Series) -> pd.Series:
    return date.apply(GetDate)

# change date column to just the day
csv_data["stamp"] = GetDateSeries(csv_data["stamp"])

csv_data.head(10)
print(csv_data.dtypes)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11404\4100932290.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_data = pd.read_csv(PATH_TO_CSV, nrows=400000, skiprows=[1])


stamp                  datetime64[ns]
yaw                           float64
pitch                         float64
roll                          float64
rotation_rate_x               float64
rotation_rate_y               float64
rotation_rate_z               float64
user_acceleration_x           float64
user_acceleration_y           float64
user_acceleration_z           float64
latitude                      float64
longitude                     float64
altitude                      float64
course                        float64
speed                         float64
horizontal_accuracy           float64
vertical_accuracy             float64
battery_state                  object
user_activity_label            object
dtype: object


In [5]:
# Gets the unique dates, and their associated latitude and longitudes
# params
#    df: The dataframe to get the dates, lat, and long from
#    date_col: The title of the date column
#    lat_col: The title of the lat column
#    long_col: The title of the long column
# returns
#    a list of tuples, where the first spot is the date, the second the latitude, and the third the longitude
def UniqueDates(df: pd.DataFrame, date_col: str, lat_col: str, long_col: str) -> list:
    unique_dates = []
    last_date = dt.datetime.min
    for i in range(len(df)):
        date = df.loc[i, date_col]
        
        # if they aren't equal, add that unique date to the list
        #next day
        if (date.date() != last_date.date()):
            
            # get the lat and long
            lat = df.loc[i, lat_col]
            long = df.loc[i, long_col]
            
            # if they are nan, move to the next available lat and long
            if (math.isnan(lat) or math.isnan(long)):
                continue
                        
            unique_dates.append((date, lat, long))
        
        last_date = date
        
    return unique_dates

In [6]:
# Gets the specified data from AirData
# params
#    email: the dataframe to add the column to
#    key: The title of the date column
#    start_date: The title of the lat column
#    end_date: The title of the long column
#    latitude: The desired latitude
#    longitude: The desired longitude
#    params: The list of integer parameters that specify the type of particulate matter you want returned
#    max_latitude: (optional) the max latitude you want to consider 
#    max_longitude: (optional) the max longitude you want to consider
# returns
#    a dictionary with the parameter name as the key, and the desired value and date as the value
# throws
#    a ValueError if the parameter list is longer than 5. AirData only permits 5 params at a time
def GetAirData(email: str, key: str, start_date: dt.datetime, end_date: dt.datetime, latitude: float, longitude: float, params: list,
               max_latitude: float = 0, max_longitude: float = 0) -> dict:
    if (len(params) > 5):
        raise ValueError("The parameter list can only contain 5 or less parameters")
    
    # create a box by adding 3 to the lat and long if they aren't passed
    if (max_latitude == 0 and max_longitude == 0):
        max_latitude = latitude + (LATITUDE_RANGE / 2)
        max_longitude = longitude + (LONGITUDE_RANGE / 2)
        latitude -= (LATITUDE_RANGE / 2)
        longitude -= (LONGITUDE_RANGE / 2)
        
        # swap max and min if they are in the wrong spots
        if (max_latitude < latitude):
            temp = max_latitude
            max_latitude = latitude
            latitude = temp
            
        if (max_longitude < longitude):
            temp = max_longitude
            max_longitude = longitude
            longitude = temp
            
    url = REQUEST_URL.format(email, key, ",".join(str(x) for x in params), str(start_date.date()).replace("-", ""), 
                             str(end_date.date()).replace("-", ""), latitude, max_latitude, longitude, max_longitude)
    print(url)
    
    response = req.get(url)
    
    if response.ok:
        data = response.json()
        values = ExtractInfo(data, latitude, longitude, start_date)
        return values
            
    else:
        response.raise_for_status()
        return None
        
# Returns a map object that has the distance to the closest stations measurement
# params
#    locations: The list of locations (the index and latitude and longitude) of the stations
#    latitude: the latitude of the measurement of the person
#    longitude: the longitude of the measurement of the person
# returns
#     a map object with the parameter name and location in a tuple
# throws
def MinimumDistPerLocation(locations: list, latitude: float, longitude: float) -> map:
    closest_params = {}
    # loop through locations
    for i, (lat, long), param in locations:
        curr_dist = hs.haversine((latitude, longitude), (lat, long))
        
        closest_params[param] = closest_params.get(param, []) + [(i, curr_dist)]
    # map the dictionary of key: lists to a dict of key:tuple, with the closest station
    return map(lambda x: (x, min(closest_params[x], key=lambda y: y[1])), closest_params)

# Extract the desired measurement from the given data
# params
#    data: the dictionary that holds the desired data
#    latitude: The desired latitude
#    longitude: The desired longitude
#    date: the date of the measurement
# returns
#    a dictionary with the parameter name as the key, and the date and measurement as its value
# throws
def ExtractInfo(data: dict, latitude: float, longitude: float, date: dt.datetime) -> dict:
    print(len(data["Data"]))
    if data["Header"][0]["rows"] == 0:
        print("No data returned")
        return
    
    locations = []
    
    # loop through each row that was returned
    for i, val in enumerate(data["Data"]):
        # skip this row if it is not valid
        if not val["validity_indicator"] == 'Y':
            continue
        
        locations.append((i, (val["latitude"], val["longitude"]), val["parameter"]))
    
    # get the closest location for each parameter
    closest_params = MinimumDistPerLocation(locations, latitude, longitude)
    arithmetic_means = {}
    for param, (index, _) in closest_params:
        arithmetic_means[param] = (data["Data"][index]["arithmetic_mean"], date)
        
    return arithmetic_means

In [15]:
# Add an the desired columns to a new dataframe
# params
#    df: the dataframe to add the column to
#    date_col: The title of the date column
#    params: the integer list of parameters to add to the dataframe
# returns
# throws
def AddParametersToData(df: pd.DataFrame, date_col: str, params: list= [PARTICULATE_MATTERS["Ozone"]]) -> pd.DataFrame:
    # get the counts of unique dates
    number_of_records = ct(df[date_col].astype(str))
    
    # retrieve the unique dates and their locations
    date_and_loc = UniqueDates(csv_data, "stamp", "latitude", "longitude")
    new_columns = {}
    
    # loop through the dates and locations and for each parameter on that day, add list to a dictionary with those values
    for date, lat, long in date_and_loc:
        values = GetAirData(AQS_DATA["email"], AQS_DATA["key"], date, date, 
                        lat, long, params)
        
        for param, (mean, date) in values.items():
            new_columns[param] = new_columns.get(param, []) + [mean] * number_of_records[str(date.date())]
            
    # add column to the new dataframe
    new_df = df.copy(deep=True)
    for param, value in new_columns.items():
        if len(value) == 0:
            continue
        new_df[param] = value
        
    return new_df
        

In [16]:
AddParametersToData(csv_data, "stamp", [44201, 42101, 42401, 42602, 85129]).head(10)

https://aqs.epa.gov/data/api/dailyData/byBox?email=chase.conaway@wsu.edu&key=sandwolf12&param=44201,42101,42401,42602,85129&bdate=20230828&edate=20230828&minlat=43.23060607910156&maxlat=50.23060607910156&minlon=-120.67095184326172&maxlon=-113.67095184326172
4
https://aqs.epa.gov/data/api/dailyData/byBox?email=chase.conaway@wsu.edu&key=sandwolf12&param=44201,42101,42401,42602,85129&bdate=20230829&edate=20230829&minlat=43.24932861328125&maxlat=50.24932861328125&minlon=-120.64724731445312&maxlon=-113.64724731445312
4


,stamp,yaw,pitch,roll,rotation_rate_x,rotation_rate_y,rotation_rate_z,user_acceleration_x,user_acceleration_y,user_acceleration_z,latitude,longitude,altitude,course,speed,horizontal_accuracy,vertical_accuracy,battery_state,user_activity_label,Ozone
0,2023-08-28,-0.033335,0.040521,1.376791,0.000530,0.000567,0.003458,-0.969708,0.053266,-0.803538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unplugged,NaN,0.0185
1,2023-08-28,0.166034,0.051798,-0.922997,-0.002101,-0.001292,-0.000076,0.806522,0.064562,-0.393836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unplugged,NaN,0.0185
2,2023-08-28,0.911395,-0.037857,0.041807,-0.001991,-0.000616,0.001076,-0.030992,-0.025427,0.001905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unplugged,NaN,0.0185
3,2023-08-28,0.910933,-0.024698,0.026261,0.002564,0.000466,0.001712,-0.015890,-0.014381,0.003470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unplugged,NaN,0.0185
4,2023-08-28,0.910843,-0.020273,0.021116,0.001488,0.002050,-0.000706,-0.010780,-0.008126,0.002089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unplugged,NaN,0.0185
5,2023-08-28,0.910789,-0.018194,0.018532,0.003596,0.004182,0.000568,-0.008198,-0.004461,0.004790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unplugged,NaN,0.0185
6,2023-08-28,0.910780,-0.016943,0.017129,0.002520,0.002087,0.000156,-0.004354,-0.004689,0.003448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unplugged,NaN,0.0185
7,2023-08-28,0.910708,-0.016079,0.016075,0.000699,0.000615,-0.001150,-0.005681,-0.003535,0.003236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unplugged,NaN,0.0185
8,2023-08-28,0.910740,-0.015435,0.015343,0.003869,0.001404,-0.001784,-0.004720,-0.002968,0.001869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unplugged,NaN,0.0185
9,2023-08-28,0.910719,-0.014973,0.014860,-0.000675,0.000058,0.001115,-0.003918,-0.002277,0.003302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unplugged,NaN,0.0185
